In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import hash_model as image_hash_model
import pickle
import time
import numpy as np

top_k = 1000
batch_size = 10
epochs = 100
lr = 0.001 #0.05
weight_decay = 10 ** -5
lambda1 = 0.01
hash_bits = 64 # 假设哈希编码的位数
model_name = "vgg11"
device = torch.device("cuda")

# 1. 加载CIFAR-10数据集
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


model = image_hash_model.HASH_Net(model_name, hash_bits)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.3, last_epoch=-1)

# 3. 加载哈希编码
with open('../labels/64_cifar10_10_class.pkl', 'rb') as f:
    label_hash_codes= torch.load(f)
label_hash_codes.to(device)

/root/anaconda3/envs/deephashh/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/root/anaconda3/envs/deephashh/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.],
        [ 1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,
         -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,
          1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,
         -1., -1.,  1.,  1., -1

In [5]:
import time
import numpy as np
import torch
import torch.nn as nn
from cac import test_accuracy
from condition import add_label_noise

noise_rate=0.1
# 初始化
start_time = time.time()
total_loss = []
accuracy_list = []  # 存储每次epoch的准确率
best_accuracy = 0.0  # 存储最高准确率

for epoch in range(epochs):  # 循环遍历数据集多次
    scheduler.step()
    if epoch % 1 == 0:
        accuracy = test_accuracy(model, testloader, label_hash_codes,device)
        print(f'Epoch {epoch}, Test Accuracy: {accuracy}%')
        accuracy_list.append(accuracy)  # 添加当前准确率到列表
        
        # 检查是否是最高准确率
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            # 保存模型
            torch.save(model.state_dict(), './model/nr_'+str(noise_rate)+'_1model.pth')
            print("Model saved with accuracy: {:.2f}%".format(best_accuracy))
    
    for iter, traindata in enumerate(trainloader, 0):
        inputs, labels = traindata
        print(labels)
        labels = add_label_noise(labels.numpy(), noise_rate=noise_rate, num_classes=10)
        labels = torch.from_numpy(labels)
        
        inputs = inputs.to(device)
        inputs = model(inputs)
        cat_codes = label_hash_codes[labels].to(device) 
        criterion = nn.BCELoss().to(device)
        center_loss = criterion(0.5*(inputs+1),0.5*(cat_codes+1))

        Q_loss = torch.mean((torch.abs(inputs)-1.0)**2)
        loss = center_loss + lambda1*Q_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss.append(loss.data.to(device).numpy)
        if iter % 500 == 0:
            end_time1 = time.time()
            print('epoch: %d, lr: %.5f iter_num: %d, time: %.3f, loss: %.3f' % 
                  (epoch, lr, iter, (end_time1-start_time), loss))
        
end_time = time.time()
epoch_loss = np.mean(total_loss)
print('Finished Training')

# 打印所有epoch的准确率
print("Epoch accuracies:", accuracy_list)


Epoch 0, Test Accuracy: 91.62%
Model saved with accuracy: 91.62%
tensor([1, 2, 6, 0, 8, 1, 9, 5, 1, 6])
epoch: 0, lr: 0.00100 iter_num: 0, time: 24.825, loss: 0.201
tensor([1, 2, 3, 3, 7, 6, 6, 1, 4, 6])
tensor([6, 4, 7, 7, 8, 3, 2, 4, 0, 9])
tensor([3, 7, 9, 1, 0, 0, 9, 5, 0, 0])
tensor([3, 9, 8, 2, 5, 0, 0, 4, 7, 0])
tensor([6, 6, 6, 3, 0, 1, 2, 5, 1, 9])
tensor([8, 7, 7, 0, 8, 8, 8, 9, 5, 0])
tensor([6, 8, 0, 9, 2, 0, 3, 8, 0, 4])
tensor([4, 9, 4, 9, 0, 3, 8, 4, 7, 9])
tensor([9, 2, 5, 5, 8, 2, 3, 9, 9, 8])
tensor([3, 6, 5, 8, 4, 6, 2, 1, 9, 7])
tensor([4, 1, 4, 9, 2, 1, 6, 0, 1, 4])
tensor([8, 6, 4, 9, 7, 9, 0, 6, 0, 9])
tensor([7, 4, 8, 8, 3, 6, 8, 9, 2, 5])
tensor([1, 6, 2, 8, 1, 6, 3, 9, 5, 9])
tensor([4, 3, 8, 4, 7, 2, 1, 6, 6, 5])
tensor([9, 6, 0, 1, 5, 6, 4, 4, 0, 6])
tensor([0, 6, 3, 5, 2, 7, 7, 9, 1, 5])
tensor([1, 5, 9, 2, 6, 6, 5, 7, 2, 2])
tensor([7, 0, 7, 0, 7, 8, 7, 9, 9, 2])
tensor([1, 0, 4, 0, 0, 4, 0, 3, 3, 2])
tensor([7, 5, 0, 3, 9, 4, 2, 2, 6, 6])
tensor([3, 3, 4,

KeyboardInterrupt: 

In [ ]:
screen -S exp 